### 그래픽 카드 설정 및 확인
- 현재 kernal에서 버전 확인이 꼭 필요한 것들 확인
- torch 버전이 2.5.1+cu121이 아닐 경우 아래의 주석 해제 후 실행

In [1]:
%pip show tensorflow
%pip show torch
%pip show transformers
%pip show mlflow

Name: tensorflow
Version: 2.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: g:\싸피\코드 관련\.venv\lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: g:\싸피\코드 관련\.venv\lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, peft, torchaudio, torchvision
Note: you may need to restart the kernel to use updated packages.
Name: transformers
Version: 4.57.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/t

In [2]:
import torch
print(torch.__version__)       # 2.5.1
print(torch.version.cuda)      # 12.1
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.5.1+cu121
12.1
True
NVIDIA GeForce RTX 2070 SUPER


토치 버전 확인

In [3]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)

g:\싸피\코드 관련\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers.training_args


CUDA 확인

In [4]:
import torch

print(torch.cuda.get_device_name(0))

#!nvidia-smi

NVIDIA GeForce RTX 2070 SUPER


### 로컬에선 GPU 지정 필요없음. GPU 서버에서는 주석 풀기

In [5]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

- 현재 파일에서는 hugginface의 beomi/KcELECTRA-base 모델을 사용할 예정입니다.
- 해당 모델에 관한 정보는 config 파일을 통해 관리합니다.
- 다른 모델을 사용하기 원할 경우 config의 MODEL_NAME을 huggingface의 가이드에 따라 변경하면 됩니다.

## 성능 향상을 위한 함수
1. Text 정제
- huggingface에서 개발자가 공개한 성능을 높이기 위한 데이터 전처리 과정입니다. 특수문자 및 이모지 등을 제거합니다
- 불용어는 제거하지 않습니다. 감정 모델에서는 불용어 또한 중요한 데이터일 수 있습니다.
2. KcELECTRA 최적화인 64 토큰으로 처리하기 위한 함수 설정
- 논문 결과에 KcELECTRA는 토큰의 길이가 64일 경우 성능이 가장 최대로 나옵니다

In [6]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

def chunk_by_tokens(text, tokenizer, max_len=64, stride=16):
    """문자열을 토큰 길이 기준으로 슬라이딩 윈도우 분할"""
    tokens = tokenizer(
        text,
        add_special_tokens=False,
        return_attention_mask=False,
        return_token_type_ids=False
    )["input_ids"]

    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_len - 2, len(tokens))
        input_ids = (
            [tokenizer.cls_token_id]
            + tokens[start:end]
            + [tokenizer.sep_token_id]
        )
        chunk_text = tokenizer.decode(input_ids, skip_special_tokens=True)
        chunks.append(chunk_text)

        if end == len(tokens):
            break
        start = end - stride  # stride 만큼 겹치게 이동
    return chunks

### 텍스트 정제 확인

In [7]:
sentence = "나는 오늘 선배한테 혼났어 ★."
cleaned_sentence = clean(sentence)
print(cleaned_sentence)

나는 오늘 선배한테 혼났어 .


#### 감정 매핑.

In [8]:
sentiment_label = {
    "happy" : "기쁨",
    "anger" : "분노",
    "unrest" : "불안",
    "embarrass" : "당황",
    "sadness" : "슬픔",
    "damaged" : "상처"
}

Emotion_Classification = list(sentiment_label.keys())

## 학습 실행

#### 1. 모델 설정을 위한 기본 세팅

In [9]:
!pip show protobuf

Name: protobuf
Version: 4.25.8
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: g:\싸피\코드 관련\.venv\lib\site-packages
Requires: 
Required-by: databricks-sdk, mlflow-skinny, mlflow-tracing, opentelemetry-proto, tensorboard, tensorflow-intel


In [4]:
import transformers
transformers.utils.move_cache()

0it [00:00, ?it/s]


In [10]:
%pip show transformers accelerate peft


Name: transformers
Version: 4.45.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: g:\싸피\코드 관련\.venv\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft
---
Name: accelerate
Version: 0.34.2
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: g:\싸피\코드 관련\.venv\lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft
---
Name: peft
Version: 0.17.1
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://

In [11]:
import os
import json
import random
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from config_train import ModelConfig as cf

model_name = cf.MODEL_NAME
# 전처리한 json 파일 절대 경로(상대도 가능할듯?)
DATA_DIR = r"G:\싸피\코드 관련\S13P31A106\ai\Data\018.감성대화"
OUTPUT_DIR = r"G:\싸피\코드 관련\S13P31A106\ai\Data\018.감성대화\model_checkpoint"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 모델 설정. 나중에도 
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(Emotion_Classification),
    problem_type="multi_label_classification"
)

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'is_grokadamw_available' from 'transformers.utils' (g:\싸피\코드 관련\.venv\lib\site-packages\transformers\utils\__init__.py)

#### 1-2. LoRA를 붙이기 위해 모델명 탐색

In [ ]:
def find_lora_target_modules(model, patterns=None, min_hits=10):
    """
    Electra 계열의 self-attention 쿼리/키/밸류/출력 dense 레이어를 자동 탐색.
    patterns: 포함시킬 하위 모듈명 키워드 리스트 (정규식 포함 가능)
    min_hits: 너무 적게 잡히면 경고용
    """
    if patterns is None:
        # Electra(HF)에서 흔히 보이는 명칭들
        patterns = [r"\.query$", r"\.key$", r"\.value$", r"\.dense$"]

    names = []
    for n, m in model.named_modules():
        if isinstance(m, torch.nn.Linear):
            for p in patterns:
                if re.search(p, n):
                    names.append(n.split(".")[-1])  # 마지막 토큰만 추출 (PEFT는 leaf 모듈명 지정 선호)
                    break

    # leaf 모듈명만 추출했기 때문에 중복 제거
    names = sorted(list(set(names)))

    # Electra 구현에 따라 self.dense / output.dense 등 혼재 -> 최소세트 보정
    if len(names) < 3:
        print("[WARN] LoRA target modules detected too few:", names)

    print("[LoRA target modules]", names)
    return names


In [ ]:
# 모델이 설정되어있는 경우에만 사용, 현재는 이렇게 되어있음.
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=8,
#     problem_type="multi_label_classification"
# )

from peft import LoraConfig, get_peft_model, PeftModel, TaskType
import mlflow
import os, re, torch

# Electra 계열 타깃 모듈 자동 탐색
target_modules = find_lora_target_modules(model)  # 대체로 ['query','key','value','dense'] 식으로 나옴

# LoRA 설정
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,   # 시퀀스 분류
    r=8,                          # 랭크 (메모리/속도/성능 절충)
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=target_modules # 위에서 탐색한 모듈에만 로라 적용
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # trainable params 점검


#### 1-3. MLflow 추적을 위한 기본 세팅
- 학습 함수의 training_args에 mlflow로 삽입은 해뒀음.
- GPU 서버에서 돌릴 경우 서버와 로컬을 연결하는 파이프라인 설정 필요.
- GPU 서버쪽에서 따로 열어보는건 추후에

In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
os.environ["MLFLOW_EXPERIMENT_NAME"] = "KcELECTRA_emotion_finetuning"

#### 2. 추론 카테고리 설정

In [ ]:
# 감정 카테고리
CATEGORIES = [
    "happy",
    "anger",
    "unrest",
    "embarrass",
    "sadness",
    "damaged"
]
NUM_LABELS = len(CATEGORIES)

#### 3. 데이터셋 로드

In [ ]:
class EmotionDataset(Dataset):
    # 논문상에선 시퀀스가 64에서 최적의 성능을 뽑아낸다고 했으나 단순 감정 학습이 아니라 상담 데이터 학습이므로 일단 128로 설정. 추후 실험 예정.
    def __init__(self, json_files, tokenizer, max_len=cf.SEQUENCE_LEN, stride=cf.SEQUENCE_STRIDE_LEN):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.stride = stride

        for path in json_files:
            with open(path, "r", encoding="utf-8-sig") as f:
                samples = json.load(f)

            for s in samples:
                if not s.get("text") or not s.get("labels"):
                    continue
                
                if type(s["text"]) == int:
                    continue
                else:
                    raw_text = clean(s["text"])
                if ('네' in raw_text or '아니오' in raw_text) and len(raw_text) <= 5:
                    continue
                labels = s["labels"]
                y = [labels.get(cat, 0) for cat in CATEGORIES]

                # 토큰 기준 분할
                for chunk_text in chunk_by_tokens(raw_text, tokenizer, max_len, stride):
                    self.data.append({"text": chunk_text, "labels": y})

        print(f"✅ Loaded {len(self.data)} total samples (after chunking)")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        enc = self.tokenizer(
            item["text"],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(item["labels"], dtype=torch.float)
        }

#### 4.학습

training_args 부분만 LoRA 적용을 위해 수정.
학습 데이터는 MLflow에서 받아볼 거임

In [ ]:
from transformers import TrainingArguments, __version__ as tfv
import inspect
print("transformers version:", tfv)
print("TrainingArguments module:", TrainingArguments.__module__)
print("TrainingArguments init signature:", inspect.signature(TrainingArguments.__init__))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(pred):
    logits, labels = pred
    # Multi-label classification용 sigmoid 적용
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)   # 0.5 기준으로 0/1 예측

    # 각종 metric 계산
    return {
        "accuracy": (preds == labels).mean(),
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
        "precision": precision_score(labels, preds, average="macro", zero_division=0),
        "recall": recall_score(labels, preds, average="macro", zero_division=0),
    }

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 파일 분할 (세션 단위)
files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith(".json")]
random.shuffle(files)
split_idx = int(len(files) * 0.8)
train_files, val_files = files[:split_idx], files[split_idx:]

train_ds = EmotionDataset(train_files, tokenizer)
val_ds = EmotionDataset(val_files, tokenizer)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,              # 기존 경로
    per_device_train_batch_size=cf.TRAIN_BATCH_SIZE,     
    per_device_eval_batch_size=cf.EVAL_BATCH_SIZE,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    learning_rate=cf.LEARNING_RATE,                 # LoRA면 3e-5 ~ 5e-5 권장 (데이터 따라 튜닝)
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",         # compute_metrics에서 f1 반환한다고 가정
    greater_is_better=True,
    fp16=True,
    logging_steps=50,
    report_to=["mlflow"],               # MLflow에서 report 받아서 확인할거임.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model(os.path.join(OUTPUT_DIR, "best_model"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "best_model"))

#### 5. 어댑터 설정 - 얘는 ONNX 등의 변환이 필요하면 쓰고 아니면 말거임.

In [ ]:
# # (1) 어댑터만 저장
# adapter_dir = os.path.join(OUTPUT_DIR, "lora_adapter")
# trainer.model.save_pretrained(adapter_dir)
# tokenizer.save_pretrained(adapter_dir)

# # 추론 시: 기반 모델 + 어댑터 로드
# # base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8, problem_type="multi_label_classification")
# # model_lora = PeftModel.from_pretrained(base_model, adapter_dir)

# # (2) 병합해서 일반 모델로 내보내기 (필요할 때)
# merged_dir = os.path.join(OUTPUT_DIR, "merged_model")
# model_lora = trainer.model
# merged = model_lora.merge_and_unload()   # LoRA 가중치를 베이스에 병합
# merged.save_pretrained(merged_dir)
# tokenizer.save_pretrained(merged_dir)


# 추론

In [ ]:
def predict_emotion(text: str):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze().tolist()
    return {CATEGORIES[i]: round(float(probs[i]), 3) for i in range(NUM_LABELS)}

print(predict_emotion("요즘 너무 무기력하고 잠도 잘 안 와요."))